# Scraping the Data
In order t

In [1]:
import requests 
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import re
import h5py
from selenium import webdriver
import time
import json
import csv

In [4]:
base_dir="https://magic.wizards.com/"
base_url="https://magic.wizards.com/en/content/deck-lists-magic-online-products-game-info"

In [7]:
# create a new chrome page. Needs chromedriver.exe to be in path. I put next to anaconda stuff
driver = webdriver.Chrome()
driver.implicitly_wait(30)
driver.get(base_url)
driver.maximize_window()

In [ ]:
#Uses filter to access only certain types of decks

driver.execute_script("window.scrollTo(0, 200)") 
Filterby=driver.find_element_by_id('event_filter')
Filterby.click()
time.sleep(1)
Modern_button=driver.find_element_by_xpath("//li[contains(text(), 'Modern')]")
Modern_button.click()
time.sleep(1)
Search_button=driver.find_element_by_id('custom-search-submit')
Search_button.click()

# select by visible text
# select.select_by_visible_text('Modern')

In [9]:
#After opening the url above, Selenium clicks the scrolls through all the see more archives
'''
scrolls to bottom of page
clicks da fish link to see more info
compares old window height with new one
if equal, continually checks for 60 seconds
if not continues to scroll to the bottom and clicks the link to see more
timeout= time willing to wait for page to load for each click of the see more button
'''

SCROLL_PAUSE_TIME = 1

timeout = 300

while True:

    # Get scroll height
    ### This is the difference. Moving this *inside* the loop
    ### means that it checks if scrollTo is still scrolling 
    last_height = driver.execute_script("return document.body.scrollHeight")

    # Scroll down to bottom
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    
    # Wait to load page
    time.sleep(SCROLL_PAUSE_TIME)
    
    # touch da fishy
    da_fishy = driver.find_element_by_class_name('see-more-article-listing-section') #See more
    da_fishy.click() #click see more

    # Calculate new scroll height and compare with last scroll height
    new_height = driver.execute_script("return document.body.scrollHeight")
    i=0
    while (new_height == last_height) and (i<timeout) :

        # Calculate new scroll height and compare with last scroll height
        new_height = driver.execute_script("return document.body.scrollHeight")

        time.sleep(1)
        i+=1
    if i==timeout:
        break



ElementNotVisibleException: Message: element not interactable
  (Session info: chrome=70.0.3538.77)
  (Driver info: chromedriver=2.43.600210 (68dcf5eebde37173d4027fa8635e332711d2874a),platform=Windows NT 6.1.7601 SP1 x86_64)


In [10]:
#Handoff to BS and find the aricles we want
soup=BeautifulSoup(driver.page_source,"html.parser")
# Opens all links and gets all data
links=[]
for link in soup.find("div",{"class":"articles-listing"}).find_all('a'):
    a=link.get('href')
    links.append(a)

#Filtertype=[i.split("/")[-1] for i in links]
len(links)

8941

In [11]:
# Save links to file
linklist=pd.DataFrame(data={"col1": links})
linklist.to_csv("./linklist.csv", sep=',',index=False)

In [49]:
with open('linklist.csv', 'r') as f:
    reader = csv.reader(f)
    links_ls = list(reader)
del links_ls[0]
links = [item for sublist in links_ls for item in sublist]

In [183]:
# Meat of the scrape
'''
Go to each link and scrape the useful info

'''

Headers=['Play Type','Deck Name', 'Card Type','Card Count', 'Card Name', 'Card Link', \
         'Sideboard Card Count', 'Sideboard Card Name']
deck_data_ls=[]
deck_name_ls=[]

for page in links[0:2]:
    page_url=base_dir+page
    r=requests.get(page_url)
    c=r.content
    page_soup=BeautifulSoup(c,"html.parser")
    Play_Type=page.split("/")[-1]
    
    for deckname in page_soup.find_all("div",{"class":"deck-group"}):
        deck_name=deckname.find("span",{"class":"deck-meta"}).h4.string
        deck_name_ls.append(deck_name)
        card_count_ls=[]
        card_name_ls=[]
        card_type=[]
        card_link_ls=[]
        sideb_count=[]
        sideb_name=[]
            
        for deck in deckname.find("div", {"class":"sorted-by-overview-container"}).find_all("div",{"class":"element"}):
            card_type.append(deck.h5.text)
            card_count=[]
            card_name=[]
            card_link=[]

            for cardc in deck.find_all("span",{"class":"card-count"}):
                card_count.append(cardc.text)

            for cardn in deck.find_all("span",{"class":"card-name"}):
                card_name.append(cardn.text)

                try:
                    card_link.append(deck.find("span",{"class":"card-name"}).a.get('href'))
                except AttributeError:
                    card_link.append(deck.find('NaN'))
                    
            card_name_ls.append(card_name)
            card_count_ls.append(card_count)
            card_link_ls.append(card_link)
        try:    
            for sideb in deckname.find("div", {"class":"sorted-by-sideboard-container"}).find_all("span",{"class":"row"}):
                sideb_count.append(sideb.find("span",{"class":"card-count"}).text)
                sideb_name.append(sideb.find("span",{"class":"card-name"}).text)
        except AttributeError:
            sideb_count.append('NaN')
            sideb_name.append('NaN')

        deck_data=dict(zip(Headers,[Play_Type,deck_name,card_type, card_count_ls,\
                                 card_name_ls, card_link_ls, sideb_count, sideb_name])) 
        deck_data_ls.append(deck_data)

competitive-modern-constructed-league-2018-11-06
competitive-modern-constructed-league-2018-11-06
competitive-modern-constructed-league-2018-11-06
competitive-modern-constructed-league-2018-11-06
competitive-modern-constructed-league-2018-11-06
competitive-modern-constructed-league-2018-11-06
competitive-modern-constructed-league-2018-11-06
competitive-modern-constructed-league-2018-11-06
competitive-modern-constructed-league-2018-11-06
competitive-modern-constructed-league-2018-11-06
competitive-modern-constructed-league-2018-11-06
competitive-modern-constructed-league-2018-11-06
competitive-modern-constructed-league-2018-11-06
competitive-modern-constructed-league-2018-11-06
competitive-modern-constructed-league-2018-11-06
competitive-modern-constructed-league-2018-11-06
competitive-modern-constructed-league-2018-11-06
competitive-modern-constructed-league-2018-11-06
competitive-modern-constructed-league-2018-11-06
competitive-modern-constructed-league-2018-11-06
competitive-modern-c

In [178]:
links[0:2]

['/en/articles/archive/mtgo-standings/competitive-modern-constructed-league-2018-11-06',
 '/en/articles/archive/mtgo-standings/sealed-grn-block-ptq-2018-11-05']

In [184]:
Data_temp=pd.DataFrame(columns=Headers)
Data_set=Data_temp.append(deck_data_ls)
Data_set

,Play Type,Deck Name,Card Type,Card Count,Card Name,Card Link,Sideboard Card Count,Sideboard Card Name
0,competitive-modern-constructed-league-2018-11-06,Melon9 (5-0),"[Creature (12), Sorcery (14), Instant (16), La...","[[4, 4, 4], [2, 4, 4, 4], [4, 4, 4, 4], [3, 3,...","[[Arclight Phoenix, Bedlam Reveler, Thing in t...",[[http://gatherer.wizards.com/Pages/Search/Def...,"[2, 2, 1, 2, 2, 4, 2]","[Abrade, Alpine Moon, Anger of the Gods, Dispe..."
1,competitive-modern-constructed-league-2018-11-06,urzatheplaneswalker (5-0),"[Creature (13), Instant (1), Enchantment (27),...","[[1, 4, 4, 4], [1], [4, 4, 1, 2, 3, 3, 4, 4, 1...","[[Dryad Arbor, Gladecover Scout, Kor Spiritdan...",[[http://gatherer.wizards.com/Pages/Search/Def...,"[1, 2, 1, 1, 3, 1, 2, 2, 2]","[Leyline of Sanctity, Path to Exile, Damping S..."
2,competitive-modern-constructed-league-2018-11-06,oivanyu29 (5-0),"[Creature (28), Instant (8), Artifact (3), Lan...","[[4, 2, 4, 4, 3, 2, 1, 4, 4], [4, 4], [3], [2,...","[[Drogskol Captain, Geist of Saint Traft, Maus...",[[http://gatherer.wizards.com/Pages/Search/Def...,"[1, 1, 2, 3, 3, 2, 1, 2]","[Disdainful Stroke, Gaddock Teeg, Knight of Au..."
3,competitive-modern-constructed-league-2018-11-06,Qel33 (5-0),"[Creature (35), Sorcery (4), Enchantment (4), ...","[[4, 2, 2, 2, 4, 3, 4, 4, 4, 2, 4], [4], [4], ...","[[Bloodghast, Bomat Courier, Goblin Bushwhacke...",[[http://gatherer.wizards.com/Pages/Search/Def...,"[1, 2, 2, 2, 2, 3, 3]","[Ancient Grudge, Assassin's Trophy, Blasphemou..."
4,competitive-modern-constructed-league-2018-11-06,midcardpromo (5-0),"[Planeswalker (4), Creature (13), Sorcery (9),...","[[3, 1], [4, 2, 4, 3], [2, 4, 3], [4, 1, 4], [...","[[Liliana of the Veil, Liliana, the Last Hope]...",[[http://gatherer.wizards.com/Pages/Search/Def...,"[1, 1, 2, 1, 3, 1, 1, 1, 1, 1, 2]","[Liliana, the Last Hope, Damnation, Damping Sp..."
5,competitive-modern-constructed-league-2018-11-06,meerkatj (5-0),"[Planeswalker (4), Creature (16), Sorcery (5),...","[[4], [4, 4, 4, 4], [4, 1], [4, 3, 4], [4, 1, ...","[[Liliana of the Veil], [Bloodbraid Elf, Dark ...",[[http://gatherer.wizards.com/Pages/Search/Def...,"[4, 3, 4, 4]","[Damnation, Damping Sphere, Leyline of the Voi..."
6,competitive-modern-constructed-league-2018-11-06,-DestricteD (5-0),"[Planeswalker (6), Creature (17), Instant (6),...","[[2, 4], [4, 4, 1, 4, 1, 1, 2], [3, 3], [3, 3,...","[[Nahiri, the Harbinger, Saheeli Rai], [Arbor ...",[[http://gatherer.wizards.com/Pages/Search/Def...,"[1, 1, 1, 1, 1, 1, 2, 2, 1, 2, 2]","[Ethersworn Canonist, Firespout, Gaddock Teeg,..."
7,competitive-modern-constructed-league-2018-11-06,Nammersquats (5-0),"[Creature (24), Sorcery (12), Instant (6), Lan...","[[4, 4, 4, 3, 4, 4, 1], [4, 4, 4], [2, 4], [3,...","[[Bloodghast, Flameblade Adept, Flamewake Phoe...",[[http://gatherer.wizards.com/Pages/Search/Def...,"[2, 3, 1, 4, 2, 3]","[Fatal Push, Ancient Grudge, Engineered Explos..."
8,competitive-modern-constructed-league-2018-11-06,Noetherium (5-0),"[Creature (15), Sorcery (9), Instant (16), Art...","[[4, 4, 3, 4], [1, 2, 2, 4], [2, 3, 2, 3, 2, 4...","[[Death's Shadow, Gurmag Angler, Snapcaster Ma...",[[http://gatherer.wizards.com/Pages/Search/Def...,"[1, 2, 1, 1, 2, 2, 4, 2]","[Stubborn Denial, Anger of the Gods, Ceremonio..."
9,competitive-modern-constructed-league-2018-11-06,OtherWhiteStuff (5-0),"[Planeswalker (10), Sorcery (20), Instant (6),...","[[2, 3, 3, 2], [2, 1, 3, 4, 4, 4, 2], [4, 2], ...","[[Gideon Jura, Gideon of the Trials, Gideon, A...",[[http://gatherer.wizards.com/Pages/Search/Def...,"[2, 1, 2, 1, 1, 1, 2, 2, 2, 1]","[Anguished Unmaking, Batterskull, Damping Sphe..."


In [196]:
a=Data_set['Card Count']
a[0]

[['4', '4', '4'],
 ['2', '4', '4', '4'],
 ['4', '4', '4', '4'],
 ['3', '3', '4', '4', '4']]

In [28]:
Filtertype=[i.split("/")[-1] for i in links]

In [100]:
a=Data_set.to_json(orient='index')

In [101]:
with open('Data_set_Modern_1.json', 'w') as outfile:
    json.dump(a,outfile,indent=4, sort_keys=True, separators=(',', ': '), ensure_ascii=False)

In [102]:
with open('Data_set_Modern_1.json', 'r') as f:
    data = json.load(f)
df1 = pd.read_json(data)

In [103]:
df1

,0,1,2,3,4,5,6,7,8,9,...,13881,13882,13883,13884,13885,13886,13887,13888,13889,13890
Card Count,"[4, 2, 4, 3]","[1, 1, 4, 2]","[4, 4, 3, 2]","[4, 4, 4, 4]","[3, 4, 2, 4, 1, 4]","[4, 4, 4, 4, 4]","[2, 4, 3, 3, 1]","[4, 4, 2, 3]","[4, 1, 2, 3, 2]","[2, 2, 4, 4]",...,"[1, 4, 1, 4, 3, 5]","[4, 3, 1, 4, 4]","[4, 4, 4, 2, 1]","[4, 2, 4, 7]","[4, 2, 3, 4]","[4, 1, 4, 4, 2]","[3, 3, 4, 2, 4, 1]","[4, 4, 3, 1]","[1, 1, 3, 2, 5]","[3, 1, 2, 4]"
Card Link,[http://gatherer.wizards.com/Pages/Search/Defa...,[http://gatherer.wizards.com/Pages/Search/Defa...,[http://gatherer.wizards.com/Pages/Search/Defa...,[http://gatherer.wizards.com/Pages/Search/Defa...,[http://gatherer.wizards.com/Pages/Search/Defa...,[http://gatherer.wizards.com/Pages/Search/Defa...,[http://gatherer.wizards.com/Pages/Search/Defa...,[http://gatherer.wizards.com/Pages/Search/Defa...,[http://gatherer.wizards.com/Pages/Search/Defa...,[http://gatherer.wizards.com/Pages/Search/Defa...,...,[http://gatherer.wizards.com/Pages/Search/Defa...,[http://gatherer.wizards.com/Pages/Search/Defa...,[http://gatherer.wizards.com/Pages/Search/Defa...,[http://gatherer.wizards.com/Pages/Search/Defa...,[http://gatherer.wizards.com/Pages/Search/Defa...,[http://gatherer.wizards.com/Pages/Search/Defa...,[http://gatherer.wizards.com/Pages/Search/Defa...,[http://gatherer.wizards.com/Pages/Search/Defa...,[http://gatherer.wizards.com/Pages/Search/Defa...,[http://gatherer.wizards.com/Pages/Search/Defa...
Card Name,"[Arclight Phoenix, Chart a Course, Fiery Tempe...","[Dryad Arbor, Path to Exile, Daybreak Coronet,...","[Drogskol Captain, Collected Company, Aether V...","[Bloodghast, Faithless Looting, Bridge from Be...","[Liliana of the Veil, Dark Confidant, Collecti...","[Liliana of the Veil, Bloodbraid Elf, Inquisit...","[Nahiri, the Harbinger, Arbor Elf, Lightning B...","[Bloodghast, Burning Inquiry, Fatal Push, Blac...","[Death's Shadow, Faithless Looting, Dismember,...","[Gideon Jura, Damnation, Fatal Push, Concealed...",...,"[Adanto Vanguard, Appeal // Authority, Blossom...","[Deeproot Elite, Baral's Expertise, Unsummon, ...","[Angel of Invention, Chart a Course, God-Phara...","[Earthshaker Khenra, Struggle // Survive, Abra...","[Bristling Hydra, Blossoming Defense, Hadana's...","[Approach of the Second Sun, Settle the Wrecka...","[Ajani Unyielding, Angel of Invention, Sram's ...","[Jadelight Ranger, Hour of Promise, Heart of K...","[Ajani Unyielding, Carnage Tyrant, Abrade, Cas...","[The Scarab God, Commit // Memory, Arguel's Bl..."
Card Type,"[Creature (12), Sorcery (14), Instant (16), La...","[Creature (13), Instant (1), Enchantment (27),...","[Creature (28), Instant (8), Artifact (3), Lan...","[Creature (35), Sorcery (4), Enchantment (4), ...","[Planeswalker (4), Creature (13), Sorcery (9),...","[Planeswalker (4), Creature (16), Sorcery (5),...","[Planeswalker (6), Creature (17), Instant (6),...","[Creature (24), Sorcery (12), Instant (6), Lan...","[Creature (15), Sorcery (9), Instant (16), Art...","[Planeswalker (10), Sorcery (20), Instant (6),...",...,"[Creature (15), Sorcery (12), Instant (1), Art...","[Creature (31), Sorcery (3), Instant (1), Ench...","[Creature (17), Sorcery (12), Artifact (4), En...","[Creature (24), Sorcery (2), Instant (9), Land...","[Creature (26), Instant (7), Enchantment (3), ...","[Sorcery (8), Instant (1), Artifact (11), Ench...","[Planeswalker (3), Creature (19), Sorcery (4),...","[Creature (22), Sorcery (7), Artifact (6), Lan...","[Planeswalker (4), Creature (21), Instant (7),...","[Creature (7), Instant (23), Enchantment (4), ..."
Deck Name,Melon9 (5-0),urzatheplaneswalker (5-0),oivanyu29 (5-0),Qel33 (5-0),midcardpromo (5-0),meerkatj (5-0),-DestricteD (5-0),Nammersquats (5-0),Noetherium (5-0),OtherWhiteStuff (5-0),...,DeepFinesse (5-0),schott (5-0),PykaPower (5-0),OafMcNamara (5-0),Jon_Ma (5-0),corith (5-0),RcL_Hallel (5-0),Joao_Andrade (5-0),Alrawn (5-0),katuo079595 (5-0)
Play Type,competitive-modern-constructed-league-2018-11-06,competitive-modern-constructed-

In [108]:
a=df1[0]

In [119]:
a[7]

['Abrade',
 'Alpine Moon',
 'Anger of the Gods',
 'Dispel',
 'Engineered Explosives',
 'Ravenous Trap',
 'Young Pyromancer']